In [1]:
!gpustat

vclab-gpuserver-57          Sun Mar 10 16:36:41 2024  515.65.01
[0] NVIDIA GeForce RTX 3090 | 30'C,   0 % |  1124 / 24576 MB | yuzhenqin(813M)
[1] NVIDIA GeForce RTX 3090 | 51'C,   0 % | 16760 / 24576 MB | yangli(3847M) yangli(3257M) yangli(3105M) yangli(3119M) yangli(3121M)
[2] NVIDIA GeForce RTX 3090 | 29'C,   0 % |  2270 / 24576 MB | yuzhenqin(1959M)
[3] NVIDIA GeForce RTX 3090 | 27'C,   0 % |  4180 / 24576 MB | yuzhenqin(3869M)
[4] NVIDIA GeForce RTX 3090 | 37'C,   0 % |  5393 / 24576 MB | yangli(1957M) yangli(3125M)
[5] NVIDIA GeForce RTX 3090 | 55'C,  16 % |  6929 / 24576 MB | yangli(3309M) yangli(3309M)
[6] NVIDIA GeForce RTX 3090 | 27'C,   0 % |   311 / 24576 MB |
[7] NVIDIA GeForce RTX 3090 | 27'C,   0 % |  2476 / 24576 MB | robotics(2165M)


In [2]:
# 在之前的蒸馏基线上，加入ABCHLN单导联的GAN，生成4倍的数据，在ABCHLN验证，在TR进行测试。

In [3]:
from models.DG_models import generate_model
from datasets import train_loader,val_loader,test_loader
import pandas as pd
import numpy as np
import ast
import matplotlib.pyplot as plt
import os
os.environ["CUDA_VISIBLE_DEVICES"] ='4'
device='cuda:0'
import wfdb
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer
from scipy import signal
from tqdm import tqdm
import math
import time
import torch
from torch import nn
from torchsummary import summary
import sklearn.metrics
import csv
import copy
from utils import *
import torch.nn.functional as F  
from collections import deque  
import random


import sys
import ast
import csv
import math
import time
import wfdb
import copy
import torch
import random
import numpy as np
import pandas as pd
from utils import *
import torch.nn as nn
from tqdm import tqdm
import sklearn.metrics
from scipy import signal
import torch.nn.init as init
from collections import deque  
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchsummary import summary
import torch.autograd as autograd
from torch.autograd import Variable
from collections import defaultdict
import torch.backends.cudnn as cudnn
from torch.optim import lr_scheduler
from torch.nn import functional as F
from models.DG_models import generate_model
from datasets import train_loader,val_loader,test_loader
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer


In [4]:
base_lr=2e-3
gan_aug=3
batch_size=128
max_lr=base_lr*batch_size/256
min_lr=max_lr/100
distill=True
DG_method_student='DG_GR'
DG_method_teacher='teacher_self_distill_leadI_GRL'
GRL_loss_beta=0.05
DGGR_smooth_eps=0.3
ts_label_beta=0.3
ts_feature_beta=0.3
train_val_test_split=[0.8,0.2,1] # 1 for inter
base_optimizer=torch.optim.Adam
loss_fn_clf=nn.CrossEntropyLoss()
base_model_student='cnn_Ag'
base_model_teacher='cnn_Ag_teacher_self_distill_leadI'
class_num=3
smote=True
pair_num_per_anchor_batch=5
CTL_margin=0.1
warm_epoch=2
total_epoch=30
CRL_queue_len=5
finetune_epoch=10
# 计算两个模型标签的余弦相似度损失

class CosineSimilarityLoss(torch.nn.Module):  
    def __init__(self):  
        super(CosineSimilarityLoss, self).__init__()  
  
    def forward(self, input1, input2):  
        cos_sim = torch.mean(F.cosine_similarity(input1, input2))
        loss = 1 - cos_sim  
        return loss

loss_fn_tslb = CosineSimilarityLoss()

# 基于欧氏距离的特征对比损失 

# 没有温度系数没考虑，因为relu已经考虑了平滑的效果。

# 如果想用温度系数建议去问大模型。
  
class ContrastiveLoss(torch.nn.Module):  
    def __init__(self, margin=1.):  
        super(ContrastiveLoss, self).__init__()  
        self.margin = torch.tensor(margin, dtype=torch.float32)
  
    def forward(self, anchor, positive, negative):  
#         print('anchor:',anchor)
#         print('positive:',positive)
#         print('negative:',negative)
        # 计算锚点和正样本之间的余弦相似度  
        similarity_anchor_positive = F.cosine_similarity(anchor, positive, dim=-1)  
#         print('positive:',similarity_anchor_positive)
        # 计算锚点和负样本之间的余弦相似度  
        similarity_anchor_negative = F.cosine_similarity(anchor, negative, dim=-1)  
#         print('negtive:',similarity_anchor_negative)
  
        # 计算损失  
        loss = torch.mean(F.relu(similarity_anchor_positive-similarity_anchor_negative))
#         print('cft_loss=',loss)
        return loss

feature_contrastive_loss_fn = ContrastiveLoss(margin=CTL_margin)

# 维护教师的特征内存队列和标签队列

class TeacherQueue:  
    def __init__(self, max_size):  
        self.max_size = max_size  
        self.feat_queue = deque(maxlen=max_size)  
        self.label_queue = deque(maxlen=max_size)  
  
    def enqueue(self, features,labels):  
        """  
        将特征向量加入队列  
        :param features: 特征向量，形状为[batch_size, feature_dim]  
        """  
        if len(self.feat_queue) == self.max_size:  
            self.dequeue()  # 如果队列已满，先出队  
        self.feat_queue.append(features)  
        self.label_queue.append(labels)  
  
    def dequeue(self):  
        """  
        从队列中移除最早加入的特征向量，无需返回值。  
        """  
        self.feat_queue.popleft()  
        self.label_queue.popleft()  
        
        return 0
    
    def get_queue_len(self):
        return len(self.feat_queue)
  
    def get_all_queue(self):  
        """  
        返回队列中所有的特征向量和标签，形状为[total_size, feature_dim],[total_size, label_dim]  
        """  
        return torch.cat(list(self.feat_queue)),torch.cat(list(self.label_queue))

def get_class_f1(tar,pred):
    # 多分类多标签或单标签，输入为概率或one hot
    pred_idx = pred.argmax(axis=1)
    pred_=(pred_idx[:,None] == np.arange(pred.shape[1])).astype(int)
    F1_normal_score,F1_AF_score,F1_other_score=list(sklearn.metrics.f1_score(tar, pred_, average=None))
    return F1_normal_score,F1_AF_score,F1_other_score


def test_student_one_epoch(epoch,warm_epoch, total_epoch, student_model,loss_fn_clf, \
                          test_datasloader, device):
    
    student_model.eval() 
    running_loss = None 
    test_pred=0
    running_loss = []
    test_tar=0
    for step, (in_data_, targets_) in (enumerate(test_datasloader)):
        tmp_data=np.zeros((in_data_.shape[0],1000,2))
        tmp_data[:,:,0]=in_data_[:,:,0]
        tmp_data[:,:-1,1]=in_data_[:,1:,0]-in_data_[:,0:-1,0]
        preds_,_,_=student_model(torch.tensor(tmp_data).float().to(device))
        preds_=preds_.float()
        targets_for_loss=torch.max(targets_[:,:class_num],1).indices.long().to(device)
        loss=loss_fn_clf(preds_,targets_for_loss)    
        running_loss.append(loss.item())
        mean_loss=np.mean(running_loss)

        # 使用AUC作为评价指标
        if type(test_pred)==int:
            test_pred=preds_.clone().detach().cpu().numpy()
        else:
            test_pred=np.concatenate((test_pred,
                                preds_.clone().detach().cpu().numpy(),
                                ),axis=0)
        if type(test_tar)==int:
            test_tar=targets_.clone().detach().cpu().numpy()
        else:
            test_tar=np.concatenate((test_tar,
                                targets_.clone().detach().cpu().numpy(),
                                ),axis=0)

    print('epoch=',epoch,)
    print("test_loss=",mean_loss)
    print("________________________________")
    mean_macro_auc=sklearn.metrics.roc_auc_score(test_tar, test_pred, average='macro')
#     print('test mean_macro_auc= ',mean_macro_auc)
    
#     print(test_tar.shape)
#     print(test_pred.shape)
    
    F1_score=get_macro_f1(test_tar,test_pred)
#     print('test F1_score= ',F1_score)
    
    precision=get_macro_precision(test_tar,test_pred)
#     print('test precision= ',precision)
    
    recall=get_macro_recall(test_tar,test_pred)
#     print('test recall= ',recall)

    acc=get_acc(test_tar,test_pred)
#     print('test acc= ',acc)
    
    F1_normal_score,F1_AF_score,F1_other_score=get_class_f1(test_tar, test_pred)
    return str(int(round(1000*mean_macro_auc))),str(int(round(1000*precision))),str(int(round(1000*recall))),str(int(round(1000*F1_score))),str(int(round(1000*F1_normal_score))),str(int(round(1000*F1_AF_score))),str(int(round(1000*F1_other_score))),str(int(round(1000*acc)))

tem_relu=nn.ReLU(inplace=False)

# 定义训练流程
def finetune_student_one_epoch(epoch,warm_epoch,total_epoch,student_model,teacher_model, \
                            max_lr,min_lr,loss_fn_clf,loss_fn_tslb,optimizer, \
                            feature_contrastive_loss_fn, \
                            Teacher_queue,base_optimizer, \
                            train_datasloader,device):
    student_model.train()
    running_loss = None 
    # 维护一个对比损失的内存队列
    for step, (in_data_, targets_) in (enumerate(tqdm(train_datasloader))):
        # 教师模型的预测
        
        assert (gan_aug==3) # 进行一次教师模型的前向传播和队列存储，一次原有单导联蒸馏，3次单导联生成蒸馏
        
        current_iter=step+epoch*len(train_datasloader)
        warm_iter=warm_epoch*len(train_datasloader)
        total_iter=total_epoch*len(train_datasloader)
        adjust_learning_rate(optimizer,current_iter,warm_iter,total_iter,max_lr,min_lr)
        
        
#         with torch.no_grad():
#             in_data_1 = in_data_[:,:,:1].float().to(device).clone()
#             in_data_2 = GAN_model.generate_images(torch.cat((in_data_1,in_data_1,in_data_1),dim=0))  # 32,

#             in_data_GAN4=torch.cat((in_data_1,in_data_2),dim=0)
                                
        optimizer.zero_grad()
        
        
        
        tmp_data=np.zeros((in_data_.shape[0],1000,2))
        tmp_data[:,:,0]=in_data_[:,:,0]
        tmp_data[:,:-1,1]=in_data_[:,1:,0]-in_data_[:,0:-1,0]
        student_diag_preds_,student_domain_preds_,student_feature=student_model(torch.tensor(tmp_data).float().to(device))
        student_diag_preds_=student_diag_preds_.float()

        # 损失函数的计算和反向传播
        # 分类损失部分
        targets_for_diag_loss=torch.max(targets_[:,:class_num],1).indices.long().to(device)  # 这里的2是指两类。
        loss_diag=loss_fn_clf(student_diag_preds_,targets_for_diag_loss)
        loss=loss_diag
        # 反向传播
        loss.backward()
        optimizer.step()
        if running_loss is None:
            running_loss = loss.item() # 只输出重构损失
        else:
            running_loss = running_loss * .9 + loss.item() * .1   

#             print('epoch=',epoch,)
    print("train_running_loss=",running_loss)

            # loss的打印
            
    # 特征对比损失部分    
    # 分类结果差异损失部分
    # domain损失部分
    # 分类损失部分





In [5]:
# 不finetune
# 加载数据

finetune_ratio=0


test_T_data_list=['T_3cls']
test_T_datasloader=test_loader(dataname=test_T_data_list,train_val_test_split=[finetune_ratio,0,1-finetune_ratio],bs=batch_size,num_workers=2,class_num=class_num).loader()

test_R_data_list=['R_3cls']
test_R_datasloader=test_loader(dataname=test_R_data_list,train_val_test_split=[finetune_ratio,0,1-finetune_ratio],bs=batch_size,num_workers=2,class_num=class_num).loader()





# 更改加载的教师模型字典：
# 开始训练流程
student_model=generate_model(base_model=base_model_student,input_channels=2, num_classes=class_num,DG_method=DG_method_student,domain_classes=6,distill=distill).to(device)
student_model.load_state_dict(torch.load('save_model_states/1709996629_ABCLNH_TR_proposed_xuesheng_cnnAg_zong_f1.pth'))  
  
# 现在，模型已经有了之前保存的参数
teacher_model=None
Teacher_queue=None

print('ok')
optimizer=generate_optimizer(base_optimizer=base_optimizer,model_parameters=student_model.parameters(),SAM_flag=False,weight_decay=1e-4)


    
                                         
T_auc,T_pre,T_rec,T_f1_test,T_f1_test_normal,T_f1_test_AF,T_f1_test_Other,T_acc=test_student_one_epoch(0,warm_epoch, total_epoch, student_model,loss_fn_clf, \
                      test_T_datasloader, device)
print(T_auc,T_pre,T_rec,T_f1_test,T_f1_test_normal,T_f1_test_AF,T_f1_test_Other,T_acc)

R_auc,R_pre,R_rec,R_f1_test,R_f1_test_normal,R_f1_test_AF,R_f1_test_Other,R_acc=test_student_one_epoch(0,warm_epoch, total_epoch, student_model,loss_fn_clf, \
                      test_R_datasloader, device)
print(R_auc,R_pre,R_rec,R_f1_test,R_f1_test_normal,R_f1_test_AF,R_f1_test_Other,R_acc)
    



# 写入csv

header=[
        'train\\test\t',
        'auc','pre','rec','f1','normal-f1','AF-f1','other-f1','acc\t',]
#         'auc','pre','rec','f1','normal-f1','AF-f1','other-f1','acc\t',]

        # round

results=[
        [
        'T',
        T_auc,T_pre,T_rec,T_f1_test,T_f1_test_normal,T_f1_test_AF,T_f1_test_Other,T_acc+'\t',
         ],
        ]

# with open('log/dist_GAN_ABCHLN_T_finetune_005_600000_20240301_total_AF_f1_box1.csv','a',encoding='utf-8')as file_obj:
#     writer=csv.writer(file_obj)
#     writer.writerow(header)
#     for result in results:
#         writer.writerow(result)
        
        
# header=[
#         'train\\test\t',
#         'auc','pre','rec','f1','normal-f1','AF-f1','other-f1','acc\t',]
# #         'auc','pre','rec','f1','normal-f1','AF-f1','other-f1','acc\t',]

#         # round

# results=[
#         [
#         'R',
#         R_auc,R_pre,R_rec,R_f1_test,R_f1_test_normal,R_f1_test_AF,R_f1_test_Other,R_acc+'\t',
#          ],
#         ]

# with open('log/dist_GAN_ABCHLN_R_finetune_600000_20240303_total_AF_f1_correct_save_exp1_box1.csv','a',encoding='utf-8')as file_obj:
#     writer=csv.writer(file_obj)
#     writer.writerow(header)
#     for result in results:
#         writer.writerow(result)
        

  


test data T_3cls
test data size : 8249
test label class :  [5076.  758. 2415.]
total test data shape =  (8249, 1000, 1)
total test label shape =  (8249, 3)
total test data class =  [5076.  758. 2415.]
test data R_3cls
test data size : 4479
test label class :  [3548.   76.  856.]
total test data shape =  (4479, 1000, 1)
total test label shape =  (4479, 3)
total test data class =  [3548.   76.  856.]
base_model=cnn_Ag
ok
epoch= 0
test_loss= 0.9485348949065575
________________________________
793 633 673 627 636 743 503 594
epoch= 0
test_loss= 0.7349713018962315
________________________________
781 572 636 587 782 560 420 681


In [6]:
# finetune=0.05
# 加载数据

finetune_ratio=0.05

train_T_finetune_datasloader_instance=train_loader(dataname=['T_3cls'],train_val_test_split=[finetune_ratio,0,1-finetune_ratio],bs=batch_size,num_workers=2,smote=smote,maple=False,class_num=class_num)
train_T_finetune_datasloader=train_T_finetune_datasloader_instance.loader()
train_R_finetune_datasloader_instance=train_loader(dataname=['R_3cls'],train_val_test_split=[finetune_ratio,0,1-finetune_ratio],bs=batch_size,num_workers=2,smote=smote,maple=False,class_num=class_num)
train_R_finetune_datasloader=train_R_finetune_datasloader_instance.loader()
 


test_T_data_list=['T_3cls']
test_T_datasloader=test_loader(dataname=test_T_data_list,train_val_test_split=[finetune_ratio,0,1-finetune_ratio],bs=batch_size,num_workers=2,class_num=class_num).loader()

test_R_data_list=['R_3cls']
test_R_datasloader=test_loader(dataname=test_R_data_list,train_val_test_split=[finetune_ratio,0,1-finetune_ratio],bs=batch_size,num_workers=2,class_num=class_num).loader()





# 更改加载的教师模型字典：
# 开始训练流程
student_model=generate_model(base_model=base_model_student,input_channels=2, num_classes=class_num,DG_method=DG_method_student,domain_classes=6,distill=distill).to(device)
student_model.load_state_dict(torch.load('save_model_states/1709996629_ABCLNH_TR_proposed_xuesheng_cnnAg_zong_f1.pth'))  
  
# 现在，模型已经有了之前保存的参数
teacher_model=None
Teacher_queue=None

print('ok')
optimizer=generate_optimizer(base_optimizer=base_optimizer,model_parameters=student_model.parameters(),SAM_flag=False,weight_decay=1e-4)


for epoch in range(finetune_epoch):
    print('_____________________________________')
    print(' ')
    time.sleep(0.1)
    finetune_student_one_epoch(epoch,warm_epoch,total_epoch,student_model,teacher_model, \
                            max_lr,min_lr,loss_fn_clf,loss_fn_tslb,optimizer, \
                            feature_contrastive_loss_fn, \
                            Teacher_queue,base_optimizer, \
                            train_T_finetune_datasloader,device)
    
                                         
T_auc,T_pre,T_rec,T_f1_test,T_f1_test_normal,T_f1_test_AF,T_f1_test_Other,T_acc=test_student_one_epoch(epoch,warm_epoch, total_epoch, student_model,loss_fn_clf, \
                      test_T_datasloader, device)
print(T_auc,T_pre,T_rec,T_f1_test,T_f1_test_normal,T_f1_test_AF,T_f1_test_Other,T_acc)

R_auc,R_pre,R_rec,R_f1_test,R_f1_test_normal,R_f1_test_AF,R_f1_test_Other,R_acc=test_student_one_epoch(epoch,warm_epoch, total_epoch, student_model,loss_fn_clf, \
                      test_R_datasloader, device)
print(R_auc,R_pre,R_rec,R_f1_test,R_f1_test_normal,R_f1_test_AF,R_f1_test_Other,R_acc)
    



# 写入csv

header=[
        'train\\test\t',
        'auc','pre','rec','f1','normal-f1','AF-f1','other-f1','acc\t',]
#         'auc','pre','rec','f1','normal-f1','AF-f1','other-f1','acc\t',]

        # round

results=[
        [
        'T'+str(finetune_ratio),
        T_auc,T_pre,T_rec,T_f1_test,T_f1_test_normal,T_f1_test_AF,T_f1_test_Other,T_acc+'\t',
         ],
        ]

with open('log/proposed_600000_finetune_ABCHLN_TR_0308_smoteTrue.csv','a',encoding='utf-8')as file_obj:
    writer=csv.writer(file_obj)
    writer.writerow(header)
    for result in results:
        writer.writerow(result)
        
        
header=[
        'train\\test\t',
        'auc','pre','rec','f1','normal-f1','AF-f1','other-f1','acc\t',]
#         'auc','pre','rec','f1','normal-f1','AF-f1','other-f1','acc\t',]

        # round

results=[
        [
        'R'+str(finetune_ratio),
        R_auc,R_pre,R_rec,R_f1_test,R_f1_test_normal,R_f1_test_AF,R_f1_test_Other,R_acc+'\t',
         ],
        ]

with open('log/proposed_600000_finetune_ABCHLN_TR_0308_smoteTrue.csv','a',encoding='utf-8')as file_obj:
    writer=csv.writer(file_obj)
    writer.writerow(header)
    for result in results:
        writer.writerow(result)
        

  


train data T_3cls
-
(412, 1000, 1)
(412,)
smoted train data size : 783
smoted train label class :  [261. 261. 261.]
train data size : 783
train label class :  [261. 261. 261. 783.]
total train data shape =  (783, 1000, 1)
total train label shape =  (783, 4)
total train data class =  [261. 261. 261. 783.]
train data R_3cls


/home/yangli/software/miniconda3/envs/ecg_20230310/lib/python3.10/site-packages/imblearn/over_sampling/_smote/base.py:336: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(


-
(223, 1000, 1)
(223,)
smoted train data size : 516
smoted train label class :  [172. 172. 172.]
train data size : 516
train label class :  [172. 172. 172. 516.]
total train data shape =  (516, 1000, 1)
total train label shape =  (516, 4)
total train data class =  [172. 172. 172. 516.]
test data T_3cls


/home/yangli/software/miniconda3/envs/ecg_20230310/lib/python3.10/site-packages/imblearn/over_sampling/_smote/base.py:336: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(


test data size : 7837
test label class :  [4815.  721. 2301.]
total test data shape =  (7837, 1000, 1)
total test label shape =  (7837, 3)
total test data class =  [4815.  721. 2301.]
test data R_3cls
test data size : 4256
test label class :  [3376.   72.  809.]
total test data shape =  (4256, 1000, 1)
total test label shape =  (4256, 3)
total test data class =  [3376.   72.  809.]
base_model=cnn_Ag
ok
_____________________________________
 


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 11.04it/s]


train_running_loss= 0.2821794821876884
_____________________________________
 


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 11.22it/s]

train_running_loss= 0.2142082584000528


epoch= 9
test_loss= 0.6785159606126047
________________________________
858 760 682 713 856 678 605 774
epoch= 9
test_loss= 0.6322946355623358
________________________________
757 624 544 575 872 471 384 782


In [7]:
# finetune=0.1
# 加载数据

finetune_ratio=0.1

train_T_finetune_datasloader_instance=train_loader(dataname=['T_3cls'],train_val_test_split=[finetune_ratio,0,1-finetune_ratio],bs=batch_size,num_workers=2,smote=smote,maple=False,class_num=class_num)
train_T_finetune_datasloader=train_T_finetune_datasloader_instance.loader()
train_R_finetune_datasloader_instance=train_loader(dataname=['R_3cls'],train_val_test_split=[finetune_ratio,0,1-finetune_ratio],bs=batch_size,num_workers=2,smote=smote,maple=False,class_num=class_num)
train_R_finetune_datasloader=train_R_finetune_datasloader_instance.loader()
 


test_T_data_list=['T_3cls']
test_T_datasloader=test_loader(dataname=test_T_data_list,train_val_test_split=[finetune_ratio,0,1-finetune_ratio],bs=batch_size,num_workers=2,class_num=class_num).loader()

test_R_data_list=['R_3cls']
test_R_datasloader=test_loader(dataname=test_R_data_list,train_val_test_split=[finetune_ratio,0,1-finetune_ratio],bs=batch_size,num_workers=2,class_num=class_num).loader()





# 更改加载的教师模型字典：
# 开始训练流程
student_model=generate_model(base_model=base_model_student,input_channels=2, num_classes=class_num,DG_method=DG_method_student,domain_classes=6,distill=distill).to(device)
student_model.load_state_dict(torch.load('save_model_states/1709996629_ABCLNH_TR_proposed_xuesheng_cnnAg_zong_f1.pth'))  
  
# 现在，模型已经有了之前保存的参数
teacher_model=None
Teacher_queue=None

print('ok')
optimizer=generate_optimizer(base_optimizer=base_optimizer,model_parameters=student_model.parameters(),SAM_flag=False,weight_decay=1e-4)


for epoch in range(finetune_epoch):
    print('_____________________________________')
    print(' ')
    time.sleep(0.1)
    finetune_student_one_epoch(epoch,warm_epoch,total_epoch,student_model,teacher_model, \
                            max_lr,min_lr,loss_fn_clf,loss_fn_tslb,optimizer, \
                            feature_contrastive_loss_fn, \
                            Teacher_queue,base_optimizer, \
                            train_T_finetune_datasloader,device)
    
                                         
T_auc,T_pre,T_rec,T_f1_test,T_f1_test_normal,T_f1_test_AF,T_f1_test_Other,T_acc=test_student_one_epoch(epoch,warm_epoch, total_epoch, student_model,loss_fn_clf, \
                      test_T_datasloader, device)
print(T_auc,T_pre,T_rec,T_f1_test,T_f1_test_normal,T_f1_test_AF,T_f1_test_Other,T_acc)

R_auc,R_pre,R_rec,R_f1_test,R_f1_test_normal,R_f1_test_AF,R_f1_test_Other,R_acc=test_student_one_epoch(epoch,warm_epoch, total_epoch, student_model,loss_fn_clf, \
                      test_R_datasloader, device)
print(R_auc,R_pre,R_rec,R_f1_test,R_f1_test_normal,R_f1_test_AF,R_f1_test_Other,R_acc)
    



# 写入csv

header=[
        'train\\test\t',
        'auc','pre','rec','f1','normal-f1','AF-f1','other-f1','acc\t',]
#         'auc','pre','rec','f1','normal-f1','AF-f1','other-f1','acc\t',]

        # round

results=[
        [
        'T'+str(finetune_ratio),
        T_auc,T_pre,T_rec,T_f1_test,T_f1_test_normal,T_f1_test_AF,T_f1_test_Other,T_acc+'\t',
         ],
        ]

with open('log/proposed_600000_finetune_ABCHLN_TR_0308_smoteTrue.csv','a',encoding='utf-8')as file_obj:
    writer=csv.writer(file_obj)
    writer.writerow(header)
    for result in results:
        writer.writerow(result)
        
        
header=[
        'train\\test\t',
        'auc','pre','rec','f1','normal-f1','AF-f1','other-f1','acc\t',]
#         'auc','pre','rec','f1','normal-f1','AF-f1','other-f1','acc\t',]

        # round

results=[
        [
        'R'+str(finetune_ratio),
        R_auc,R_pre,R_rec,R_f1_test,R_f1_test_normal,R_f1_test_AF,R_f1_test_Other,R_acc+'\t',
         ],
        ]

with open('log/proposed_600000_finetune_ABCHLN_TR_0308_smoteTrue.csv','a',encoding='utf-8')as file_obj:
    writer=csv.writer(file_obj)
    writer.writerow(header)
    for result in results:
        writer.writerow(result)
        

  


train data T_3cls
-
(824, 1000, 1)
(824,)
smoted train data size : 1527
smoted train label class :  [509. 509. 509.]
train data size : 1527
train label class :  [ 509.  509.  509. 1527.]
total train data shape =  (1527, 1000, 1)
total train label shape =  (1527, 4)
total train data class =  [ 509.  509.  509. 1527.]
train data R_3cls


/home/yangli/software/miniconda3/envs/ecg_20230310/lib/python3.10/site-packages/imblearn/over_sampling/_smote/base.py:336: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(


-
(447, 1000, 1)
(447,)
smoted train data size : 1029
smoted train label class :  [343. 343. 343.]
train data size : 1029
train label class :  [ 343.  343.  343. 1029.]
total train data shape =  (1029, 1000, 1)
total train label shape =  (1029, 4)
total train data class =  [ 343.  343.  343. 1029.]
test data T_3cls


/home/yangli/software/miniconda3/envs/ecg_20230310/lib/python3.10/site-packages/imblearn/over_sampling/_smote/base.py:336: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(


test data size : 7425
test label class :  [4567.  676. 2182.]
total test data shape =  (7425, 1000, 1)
total test label shape =  (7425, 3)
total test data class =  [4567.  676. 2182.]
test data R_3cls
test data size : 4032
test label class :  [3205.   71.  757.]
total test data shape =  (4032, 1000, 1)
total test label shape =  (4032, 3)
total test data class =  [3205.   71.  757.]
base_model=cnn_Ag
ok
_____________________________________
 


...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 13.37it/s]


train_running_loss= 0.18577330615288698
_____________________________________
 


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 12.48it/s]

train_running_loss= 0.18381027576069314


epoch= 9
test_loss= 0.5966125049833524
________________________________
872 769 723 742 862 743 621 787
epoch= 9
test_loss= 0.637805949896574
________________________________
742 627 553 581 883 484 377 797


In [8]:
# finetune=0.2
# 加载数据

finetune_ratio=0.2

train_T_finetune_datasloader_instance=train_loader(dataname=['T_3cls'],train_val_test_split=[finetune_ratio,0,1-finetune_ratio],bs=batch_size,num_workers=2,smote=smote,maple=False,class_num=class_num)
train_T_finetune_datasloader=train_T_finetune_datasloader_instance.loader()
train_R_finetune_datasloader_instance=train_loader(dataname=['R_3cls'],train_val_test_split=[finetune_ratio,0,1-finetune_ratio],bs=batch_size,num_workers=2,smote=smote,maple=False,class_num=class_num)
train_R_finetune_datasloader=train_R_finetune_datasloader_instance.loader()
 


test_T_data_list=['T_3cls']
test_T_datasloader=test_loader(dataname=test_T_data_list,train_val_test_split=[finetune_ratio,0,1-finetune_ratio],bs=batch_size,num_workers=2,class_num=class_num).loader()

test_R_data_list=['R_3cls']
test_R_datasloader=test_loader(dataname=test_R_data_list,train_val_test_split=[finetune_ratio,0,1-finetune_ratio],bs=batch_size,num_workers=2,class_num=class_num).loader()





# 更改加载的教师模型字典：
# 开始训练流程
student_model=generate_model(base_model=base_model_student,input_channels=2, num_classes=class_num,DG_method=DG_method_student,domain_classes=6,distill=distill).to(device)
student_model.load_state_dict(torch.load('save_model_states/1709996629_ABCLNH_TR_proposed_xuesheng_cnnAg_zong_f1.pth'))  
  
# 现在，模型已经有了之前保存的参数
teacher_model=None
Teacher_queue=None

print('ok')
optimizer=generate_optimizer(base_optimizer=base_optimizer,model_parameters=student_model.parameters(),SAM_flag=False,weight_decay=1e-4)


for epoch in range(finetune_epoch):
    print('_____________________________________')
    print(' ')
    time.sleep(0.1)
    finetune_student_one_epoch(epoch,warm_epoch,total_epoch,student_model,teacher_model, \
                            max_lr,min_lr,loss_fn_clf,loss_fn_tslb,optimizer, \
                            feature_contrastive_loss_fn, \
                            Teacher_queue,base_optimizer, \
                            train_T_finetune_datasloader,device)
    
                                         
T_auc,T_pre,T_rec,T_f1_test,T_f1_test_normal,T_f1_test_AF,T_f1_test_Other,T_acc=test_student_one_epoch(epoch,warm_epoch, total_epoch, student_model,loss_fn_clf, \
                      test_T_datasloader, device)
print(T_auc,T_pre,T_rec,T_f1_test,T_f1_test_normal,T_f1_test_AF,T_f1_test_Other,T_acc)

R_auc,R_pre,R_rec,R_f1_test,R_f1_test_normal,R_f1_test_AF,R_f1_test_Other,R_acc=test_student_one_epoch(epoch,warm_epoch, total_epoch, student_model,loss_fn_clf, \
                      test_R_datasloader, device)
print(R_auc,R_pre,R_rec,R_f1_test,R_f1_test_normal,R_f1_test_AF,R_f1_test_Other,R_acc)
    



# 写入csv

header=[
        'train\\test\t',
        'auc','pre','rec','f1','normal-f1','AF-f1','other-f1','acc\t',]
#         'auc','pre','rec','f1','normal-f1','AF-f1','other-f1','acc\t',]

        # round

results=[
        [
        'T'+str(finetune_ratio),
        T_auc,T_pre,T_rec,T_f1_test,T_f1_test_normal,T_f1_test_AF,T_f1_test_Other,T_acc+'\t',
         ],
        ]

with open('log/proposed_600000_finetune_ABCHLN_TR_0308_smoteTrue.csv','a',encoding='utf-8')as file_obj:
    writer=csv.writer(file_obj)
    writer.writerow(header)
    for result in results:
        writer.writerow(result)
        
        
header=[
        'train\\test\t',
        'auc','pre','rec','f1','normal-f1','AF-f1','other-f1','acc\t',]
#         'auc','pre','rec','f1','normal-f1','AF-f1','other-f1','acc\t',]

        # round

results=[
        [
        'R'+str(finetune_ratio),
        R_auc,R_pre,R_rec,R_f1_test,R_f1_test_normal,R_f1_test_AF,R_f1_test_Other,R_acc+'\t',
         ],
        ]

with open('log/proposed_600000_finetune_ABCHLN_TR_0308_smoteTrue.csv','a',encoding='utf-8')as file_obj:
    writer=csv.writer(file_obj)
    writer.writerow(header)
    for result in results:
        writer.writerow(result)
        

  


train data T_3cls
-
(1649, 1000, 1)
(1649,)
smoted train data size : 2988
smoted train label class :  [996. 996. 996.]
train data size : 2988
train label class :  [ 996.  996.  996. 2988.]
total train data shape =  (2988, 1000, 1)
total train label shape =  (2988, 4)
total train data class =  [ 996.  996.  996. 2988.]
train data R_3cls


/home/yangli/software/miniconda3/envs/ecg_20230310/lib/python3.10/site-packages/imblearn/over_sampling/_smote/base.py:336: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(


-
(895, 1000, 1)
(895,)
smoted train data size : 2079
smoted train label class :  [693. 693. 693.]
train data size : 2079
train label class :  [ 693.  693.  693. 2079.]
total train data shape =  (2079, 1000, 1)
total train label shape =  (2079, 4)
total train data class =  [ 693.  693.  693. 2079.]
test data T_3cls


/home/yangli/software/miniconda3/envs/ecg_20230310/lib/python3.10/site-packages/imblearn/over_sampling/_smote/base.py:336: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(


test data size : 6600
test label class :  [4080.  585. 1935.]
total test data shape =  (6600, 1000, 1)
total test label shape =  (6600, 3)
total test data class =  [4080.  585. 1935.]
test data R_3cls
test data size : 3584
test label class :  [2855.   62.  668.]
total test data shape =  (3584, 1000, 1)
total test label shape =  (3584, 3)
total test data class =  [2855.   62.  668.]
base_model=cnn_Ag
ok
_____________________________________
 


...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:01<00:00, 14.94it/s]


train_running_loss= 0.23278360170893253
_____________________________________
 


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:01<00:00, 15.26it/s]

train_running_loss= 0.18251166137867275


epoch= 9
test_loss= 0.5851485505700111
________________________________
881 759 746 751 863 753 638 791
epoch= 9
test_loss= 0.6189992438469615
________________________________
789 653 594 618 882 584 389 797


In [9]:
# finetune=0.3
# 加载数据

finetune_ratio=0.3

train_T_finetune_datasloader_instance=train_loader(dataname=['T_3cls'],train_val_test_split=[finetune_ratio,0,1-finetune_ratio],bs=batch_size,num_workers=2,smote=smote,maple=False,class_num=class_num)
train_T_finetune_datasloader=train_T_finetune_datasloader_instance.loader()
train_R_finetune_datasloader_instance=train_loader(dataname=['R_3cls'],train_val_test_split=[finetune_ratio,0,1-finetune_ratio],bs=batch_size,num_workers=2,smote=smote,maple=False,class_num=class_num)
train_R_finetune_datasloader=train_R_finetune_datasloader_instance.loader()
 


test_T_data_list=['T_3cls']
test_T_datasloader=test_loader(dataname=test_T_data_list,train_val_test_split=[finetune_ratio,0,1-finetune_ratio],bs=batch_size,num_workers=2,class_num=class_num).loader()

test_R_data_list=['R_3cls']
test_R_datasloader=test_loader(dataname=test_R_data_list,train_val_test_split=[finetune_ratio,0,1-finetune_ratio],bs=batch_size,num_workers=2,class_num=class_num).loader()





# 更改加载的教师模型字典：
# 开始训练流程
student_model=generate_model(base_model=base_model_student,input_channels=2, num_classes=class_num,DG_method=DG_method_student,domain_classes=6,distill=distill).to(device)
student_model.load_state_dict(torch.load('save_model_states/1709996629_ABCLNH_TR_proposed_xuesheng_cnnAg_zong_f1.pth'))  
  
# 现在，模型已经有了之前保存的参数
teacher_model=None
Teacher_queue=None

print('ok')
optimizer=generate_optimizer(base_optimizer=base_optimizer,model_parameters=student_model.parameters(),SAM_flag=False,weight_decay=1e-4)


for epoch in range(finetune_epoch):
    print('_____________________________________')
    print(' ')
    time.sleep(0.1)
    finetune_student_one_epoch(epoch,warm_epoch,total_epoch,student_model,teacher_model, \
                            max_lr,min_lr,loss_fn_clf,loss_fn_tslb,optimizer, \
                            feature_contrastive_loss_fn, \
                            Teacher_queue,base_optimizer, \
                            train_T_finetune_datasloader,device)
    
                                         
T_auc,T_pre,T_rec,T_f1_test,T_f1_test_normal,T_f1_test_AF,T_f1_test_Other,T_acc=test_student_one_epoch(epoch,warm_epoch, total_epoch, student_model,loss_fn_clf, \
                      test_T_datasloader, device)
print(T_auc,T_pre,T_rec,T_f1_test,T_f1_test_normal,T_f1_test_AF,T_f1_test_Other,T_acc)

R_auc,R_pre,R_rec,R_f1_test,R_f1_test_normal,R_f1_test_AF,R_f1_test_Other,R_acc=test_student_one_epoch(epoch,warm_epoch, total_epoch, student_model,loss_fn_clf, \
                      test_R_datasloader, device)
print(R_auc,R_pre,R_rec,R_f1_test,R_f1_test_normal,R_f1_test_AF,R_f1_test_Other,R_acc)
    



# 写入csv

header=[
        'train\\test\t',
        'auc','pre','rec','f1','normal-f1','AF-f1','other-f1','acc\t',]
#         'auc','pre','rec','f1','normal-f1','AF-f1','other-f1','acc\t',]

        # round

results=[
        [
        'T'+str(finetune_ratio),
        T_auc,T_pre,T_rec,T_f1_test,T_f1_test_normal,T_f1_test_AF,T_f1_test_Other,T_acc+'\t',
         ],
        ]

with open('log/proposed_600000_finetune_ABCHLN_TR_0308_smoteTrue.csv','a',encoding='utf-8')as file_obj:
    writer=csv.writer(file_obj)
    writer.writerow(header)
    for result in results:
        writer.writerow(result)
        
        
header=[
        'train\\test\t',
        'auc','pre','rec','f1','normal-f1','AF-f1','other-f1','acc\t',]
#         'auc','pre','rec','f1','normal-f1','AF-f1','other-f1','acc\t',]

        # round

results=[
        [
        'R'+str(finetune_ratio),
        R_auc,R_pre,R_rec,R_f1_test,R_f1_test_normal,R_f1_test_AF,R_f1_test_Other,R_acc+'\t',
         ],
        ]

with open('log/proposed_600000_finetune_ABCHLN_TR_0308_smoteTrue.csv','a',encoding='utf-8')as file_obj:
    writer=csv.writer(file_obj)
    writer.writerow(header)
    for result in results:
        writer.writerow(result)
        

  


train data T_3cls
-
(2474, 1000, 1)
(2474,)
smoted train data size : 4551
smoted train label class :  [1517. 1517. 1517.]
train data size : 4551
train label class :  [1517. 1517. 1517. 4551.]


/home/yangli/software/miniconda3/envs/ecg_20230310/lib/python3.10/site-packages/imblearn/over_sampling/_smote/base.py:336: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(


total train data shape =  (4551, 1000, 1)
total train label shape =  (4551, 4)
total train data class =  [1517. 1517. 1517. 4551.]
train data R_3cls
-
(1343, 1000, 1)
(1343,)
smoted train data size : 3165
smoted train label class :  [1055. 1055. 1055.]
train data size : 3165
train label class :  [1055. 1055. 1055. 3165.]
total train data shape =  (3165, 1000, 1)
total train label shape =  (3165, 4)
total train data class =  [1055. 1055. 1055. 3165.]
test data T_3cls


/home/yangli/software/miniconda3/envs/ecg_20230310/lib/python3.10/site-packages/imblearn/over_sampling/_smote/base.py:336: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(


test data size : 5775
test label class :  [3559.  498. 1718.]
total test data shape =  (5775, 1000, 1)
total test label shape =  (5775, 3)
total test data class =  [3559.  498. 1718.]
test data R_3cls
test data size : 3136
test label class :  [2493.   49.  595.]
total test data shape =  (3136, 1000, 1)
total test label shape =  (3136, 3)
total test data class =  [2493.   49.  595.]
base_model=cnn_Ag
ok
_____________________________________
 


...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 36/36 [00:02<00:00, 16.32it/s]


train_running_loss= 0.2115262026362988
_____________________________________
 


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 36/36 [00:02<00:00, 15.61it/s]

train_running_loss= 0.20812515468536918


epoch= 9
test_loss= 0.5821209893278454
________________________________
881 779 752 763 873 763 652 804
epoch= 9
test_loss= 0.6412942862510681
________________________________
761 625 582 595 888 515 382 805


In [10]:
# finetune=0.5
# 加载数据

finetune_ratio=0.5

train_T_finetune_datasloader_instance=train_loader(dataname=['T_3cls'],train_val_test_split=[finetune_ratio,0,1-finetune_ratio],bs=batch_size,num_workers=2,smote=smote,maple=False,class_num=class_num)
train_T_finetune_datasloader=train_T_finetune_datasloader_instance.loader()
train_R_finetune_datasloader_instance=train_loader(dataname=['R_3cls'],train_val_test_split=[finetune_ratio,0,1-finetune_ratio],bs=batch_size,num_workers=2,smote=smote,maple=False,class_num=class_num)
train_R_finetune_datasloader=train_R_finetune_datasloader_instance.loader()
 


test_T_data_list=['T_3cls']
test_T_datasloader=test_loader(dataname=test_T_data_list,train_val_test_split=[finetune_ratio,0,1-finetune_ratio],bs=batch_size,num_workers=2,class_num=class_num).loader()

test_R_data_list=['R_3cls']
test_R_datasloader=test_loader(dataname=test_R_data_list,train_val_test_split=[finetune_ratio,0,1-finetune_ratio],bs=batch_size,num_workers=2,class_num=class_num).loader()





# 更改加载的教师模型字典：
# 开始训练流程
student_model=generate_model(base_model=base_model_student,input_channels=2, num_classes=class_num,DG_method=DG_method_student,domain_classes=6,distill=distill).to(device)
student_model.load_state_dict(torch.load('save_model_states/1709996629_ABCLNH_TR_proposed_xuesheng_cnnAg_zong_f1.pth'))  
  
# 现在，模型已经有了之前保存的参数
teacher_model=None
Teacher_queue=None

print('ok')
optimizer=generate_optimizer(base_optimizer=base_optimizer,model_parameters=student_model.parameters(),SAM_flag=False,weight_decay=1e-4)


for epoch in range(finetune_epoch):
    print('_____________________________________')
    print(' ')
    time.sleep(0.1)
    finetune_student_one_epoch(epoch,warm_epoch,total_epoch,student_model,teacher_model, \
                            max_lr,min_lr,loss_fn_clf,loss_fn_tslb,optimizer, \
                            feature_contrastive_loss_fn, \
                            Teacher_queue,base_optimizer, \
                            train_T_finetune_datasloader,device)
    
                                         
T_auc,T_pre,T_rec,T_f1_test,T_f1_test_normal,T_f1_test_AF,T_f1_test_Other,T_acc=test_student_one_epoch(epoch,warm_epoch, total_epoch, student_model,loss_fn_clf, \
                      test_T_datasloader, device)
print(T_auc,T_pre,T_rec,T_f1_test,T_f1_test_normal,T_f1_test_AF,T_f1_test_Other,T_acc)

R_auc,R_pre,R_rec,R_f1_test,R_f1_test_normal,R_f1_test_AF,R_f1_test_Other,R_acc=test_student_one_epoch(epoch,warm_epoch, total_epoch, student_model,loss_fn_clf, \
                      test_R_datasloader, device)
print(R_auc,R_pre,R_rec,R_f1_test,R_f1_test_normal,R_f1_test_AF,R_f1_test_Other,R_acc)
    



# 写入csv

header=[
        'train\\test\t',
        'auc','pre','rec','f1','normal-f1','AF-f1','other-f1','acc\t',]
#         'auc','pre','rec','f1','normal-f1','AF-f1','other-f1','acc\t',]

        # round

results=[
        [
        'T'+str(finetune_ratio),
        T_auc,T_pre,T_rec,T_f1_test,T_f1_test_normal,T_f1_test_AF,T_f1_test_Other,T_acc+'\t',
         ],
        ]

with open('log/proposed_600000_finetune_ABCHLN_TR_0308_smoteTrue.csv','a',encoding='utf-8')as file_obj:
    writer=csv.writer(file_obj)
    writer.writerow(header)
    for result in results:
        writer.writerow(result)
        
        
header=[
        'train\\test\t',
        'auc','pre','rec','f1','normal-f1','AF-f1','other-f1','acc\t',]
#         'auc','pre','rec','f1','normal-f1','AF-f1','other-f1','acc\t',]

        # round

results=[
        [
        'R'+str(finetune_ratio),
        R_auc,R_pre,R_rec,R_f1_test,R_f1_test_normal,R_f1_test_AF,R_f1_test_Other,R_acc+'\t',
         ],
        ]

with open('log/proposed_600000_finetune_ABCHLN_TR_0308_smoteTrue.csv','a',encoding='utf-8')as file_obj:
    writer=csv.writer(file_obj)
    writer.writerow(header)
    for result in results:
        writer.writerow(result)
        

  


train data T_3cls
-
(4124, 1000, 1)
(4124,)


/home/yangli/software/miniconda3/envs/ecg_20230310/lib/python3.10/site-packages/imblearn/over_sampling/_smote/base.py:336: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(


smoted train data size : 7716
smoted train label class :  [2572. 2572. 2572.]
train data size : 7716
train label class :  [2572. 2572. 2572. 7716.]
total train data shape =  (7716, 1000, 1)
total train label shape =  (7716, 4)
total train data class =  [2572. 2572. 2572. 7716.]
train data R_3cls
-
(2239, 1000, 1)
(2239,)
smoted train data size : 5319
smoted train label class :  [1773. 1773. 1773.]
train data size : 5319
train label class :  [1773. 1773. 1773. 5319.]
total train data shape =  (5319, 1000, 1)
total train label shape =  (5319, 4)
total train data class =  [1773. 1773. 1773. 5319.]
test data T_3cls


/home/yangli/software/miniconda3/envs/ecg_20230310/lib/python3.10/site-packages/imblearn/over_sampling/_smote/base.py:336: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(


test data size : 4125
test label class :  [2504.  353. 1268.]
total test data shape =  (4125, 1000, 1)
total test label shape =  (4125, 3)
total test data class =  [2504.  353. 1268.]
test data R_3cls
test data size : 2240
test label class :  [1775.   43.  423.]
total test data shape =  (2240, 1000, 1)
total test label shape =  (2240, 3)
total test data class =  [1775.   43.  423.]
base_model=cnn_Ag
ok
_____________________________________
 


...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 61/61 [00:03<00:00, 17.08it/s]


train_running_loss= 0.21454747182190514
_____________________________________
 


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 61/61 [00:03<00:00, 16.98it/s]

train_running_loss= 0.20359892995450474


epoch= 9
test_loss= 0.6173498061570254
________________________________
873 773 734 751 854 758 641 785
epoch= 9
test_loss= 0.6386434584856033
________________________________
781 677 564 606 879 571 368 792


In [11]:
# # finetune=0.8
# # 加载数据

# finetune_ratio=0.8

# train_T_finetune_datasloader_instance=train_loader(dataname=['T_3cls'],train_val_test_split=[finetune_ratio,0,1-finetune_ratio],bs=batch_size,num_workers=2,smote=smote,maple=False,class_num=class_num)
# train_T_finetune_datasloader=train_T_finetune_datasloader_instance.loader()
# train_R_finetune_datasloader_instance=train_loader(dataname=['R_3cls'],train_val_test_split=[finetune_ratio,0,1-finetune_ratio],bs=batch_size,num_workers=2,smote=smote,maple=False,class_num=class_num)
# train_R_finetune_datasloader=train_R_finetune_datasloader_instance.loader()
 


# test_T_data_list=['T_3cls']
# test_T_datasloader=test_loader(dataname=test_T_data_list,train_val_test_split=[finetune_ratio,0,1-finetune_ratio],bs=batch_size,num_workers=2,class_num=class_num).loader()

# test_R_data_list=['R_3cls']
# test_R_datasloader=test_loader(dataname=test_R_data_list,train_val_test_split=[finetune_ratio,0,1-finetune_ratio],bs=batch_size,num_workers=2,class_num=class_num).loader()





# # 更改加载的教师模型字典：
# # 开始训练流程
# student_model=generate_model(base_model=base_model_student,input_channels=2, num_classes=class_num,DG_method=DG_method_student,domain_classes=6,distill=distill).to(device)
# student_model.load_state_dict(torch.load('save_model_states/1709996629_ABCLNH_TR_proposed_xuesheng_cnnAg_zong_f1.pth'))  
  
# # 现在，模型已经有了之前保存的参数
# teacher_model=None
# Teacher_queue=None

# print('ok')
# optimizer=generate_optimizer(base_optimizer=base_optimizer,model_parameters=student_model.parameters(),SAM_flag=False,weight_decay=1e-4)


# for epoch in range(finetune_epoch):
#     print('_____________________________________')
#     print(' ')
#     time.sleep(0.1)
#     finetune_student_one_epoch(epoch,warm_epoch,total_epoch,student_model,teacher_model, \
#                             max_lr,min_lr,loss_fn_clf,loss_fn_tslb,optimizer, \
#                             feature_contrastive_loss_fn, \
#                             Teacher_queue,base_optimizer, \
#                             train_T_finetune_datasloader,device)
    
                                         
# T_auc,T_pre,T_rec,T_f1_test,T_f1_test_normal,T_f1_test_AF,T_f1_test_Other,T_acc=test_student_one_epoch(epoch,warm_epoch, total_epoch, student_model,loss_fn_clf, \
#                       test_T_datasloader, device)
# print(T_auc,T_pre,T_rec,T_f1_test,T_f1_test_normal,T_f1_test_AF,T_f1_test_Other,T_acc)

# R_auc,R_pre,R_rec,R_f1_test,R_f1_test_normal,R_f1_test_AF,R_f1_test_Other,R_acc=test_student_one_epoch(epoch,warm_epoch, total_epoch, student_model,loss_fn_clf, \
#                       test_R_datasloader, device)
# print(R_auc,R_pre,R_rec,R_f1_test,R_f1_test_normal,R_f1_test_AF,R_f1_test_Other,R_acc)
    



# # 写入csv

# header=[
#         'train\\test\t',
#         'auc','pre','rec','f1','normal-f1','AF-f1','other-f1','acc\t',]
# #         'auc','pre','rec','f1','normal-f1','AF-f1','other-f1','acc\t',]

#         # round

# results=[
#         [
#         'T'+str(finetune_ratio),
#         T_auc,T_pre,T_rec,T_f1_test,T_f1_test_normal,T_f1_test_AF,T_f1_test_Other,T_acc+'\t',
#          ],
#         ]

# with open('log/proposed_600000_finetune_ABCHLN_TR_0308.csv','a',encoding='utf-8')as file_obj:
#     writer=csv.writer(file_obj)
#     writer.writerow(header)
#     for result in results:
#         writer.writerow(result)
        
        
# header=[
#         'train\\test\t',
#         'auc','pre','rec','f1','normal-f1','AF-f1','other-f1','acc\t',]
# #         'auc','pre','rec','f1','normal-f1','AF-f1','other-f1','acc\t',]

#         # round

# results=[
#         [
#         'R'+str(finetune_ratio),
#         R_auc,R_pre,R_rec,R_f1_test,R_f1_test_normal,R_f1_test_AF,R_f1_test_Other,R_acc+'\t',
#          ],
#         ]

# with open('log/proposed_600000_finetune_ABCHLN_TR_0308.csv','a',encoding='utf-8')as file_obj:
#     writer=csv.writer(file_obj)
#     writer.writerow(header)
#     for result in results:
#         writer.writerow(result)
        

  


In [12]:
from imblearn.over_sampling import SMOTE  
  
help(SMOTE)

Help on class SMOTE in module imblearn.over_sampling._smote.base:

class SMOTE(BaseSMOTE)
 |  SMOTE(*, sampling_strategy='auto', random_state=None, k_neighbors=5, n_jobs=None)
 |  
 |  Class to perform over-sampling using SMOTE.
 |  
 |  This object is an implementation of SMOTE - Synthetic Minority
 |  Over-sampling Technique as presented in [1]_.
 |  
 |  Read more in the :ref:`User Guide <smote_adasyn>`.
 |  
 |  Parameters
 |  ----------
 |  sampling_strategy : float, str, dict or callable, default='auto'
 |      Sampling information to resample the data set.
 |  
 |      - When ``float``, it corresponds to the desired ratio of the number of
 |        samples in the minority class over the number of samples in the
 |        majority class after resampling. Therefore, the ratio is expressed as
 |        :math:`\alpha_{os} = N_{rm} / N_{M}` where :math:`N_{rm}` is the
 |        number of samples in the minority class after resampling and
 |        :math:`N_{M}` is the number of sampl